# ES Salesforce vs OSS Order Analysis

This notebook compares ES Salesforce Orders against OSS to determine which orders are **actually actively billing** and need to be migrated.

## Purpose
Validate that SF "Active" orders are actually billing in OSS. If not actively billing in OSS, they don't need to be migrated.

## Logic for "Actively Billing" in OSS
- `order_state_cd` = 'CL' (Closed = order complete, service is live)
- `bill_start_date` IS NOT NULL and <= today
- `bill_end_date` IS NULL OR > today

## SF Order Criteria
- Status IN ('Activated', 'Disconnect in Progress', 'Suspended (Late Payment)')
- Excludes orders where `Project_Group__c` contains 'PA MARKET DECOM'

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, date
from collections import defaultdict

print(f"Python: {sys.executable}")
print(f"Run started: {datetime.now()}")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Run started: 2026-01-07 17:44:32.348908


In [2]:
# === CONFIGURATION ===

# ES (Source) Salesforce Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # 'login' for production

# OSS Database Credentials
OSS_HOST = "pg01.comlink.net"
OSS_PORT = "5432"
OSS_DB = "GLC"
OSS_USER = "oss_server"
OSS_PASSWORD = "3wU3uB28X?!r2?@ebrUg"

# Active Order Status Filter
ACTIVE_STATUSES = ["Activated", "Disconnect in Progress", "Suspended (Late Payment)"]

# Exclusion filter
EXCLUDE_PROJECT_GROUP = "PA MARKET DECOM"

# OSS Order States Reference
OSS_ORDER_STATES = {
    "CL": "Closed (Active/Billing)",
    "OA": "Accepted",
    "OS": "Submitted",
    "OC": "Created",
    "PN": "Pending",
    "CA": "Cancelled",
    "OR": "Rejected",
    "OV": "Validated (Disabled)",
}

print("Configuration loaded.")
print(f"SF Active Statuses: {ACTIVE_STATUSES}")
print(f"Excluding: {EXCLUDE_PROJECT_GROUP}")

Configuration loaded.
SF Active Statuses: ['Activated', 'Disconnect in Progress', 'Suspended (Late Payment)']
Excluding: PA MARKET DECOM


In [3]:
# === CONNECT TO ES SALESFORCE ===

print("=" * 80)
print("CONNECTING TO ES SALESFORCE")
print("=" * 80)

print("\n🔌 Connecting to ES Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

CONNECTING TO ES SALESFORCE

🔌 Connecting to ES Salesforce...
✅ Connected to ES: everstream.my.salesforce.com


In [4]:
# === CONNECT TO OSS DATABASE ===

print("=" * 80)
print("CONNECTING TO OSS DATABASE")
print("=" * 80)

print("\n🔌 Connecting to OSS...")
oss_conn = psycopg2.connect(
    dbname=OSS_DB,
    user=OSS_USER,
    password=OSS_PASSWORD,
    host=OSS_HOST,
    port=OSS_PORT,
)
print(f"✅ Connected to OSS: {OSS_HOST}/{OSS_DB}")

CONNECTING TO OSS DATABASE

🔌 Connecting to OSS...
✅ Connected to OSS: pg01.comlink.net/GLC


In [5]:
# === STEP 1: QUERY SF ORDERS ===

print("\n" + "=" * 80)
print("STEP 1: QUERYING ES SALESFORCE ORDERS")
print("=" * 80)

status_filter = "','".join(ACTIVE_STATUSES)

orders_query = f"""
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Billing_Invoice__r.Name,
    OSS_Order__c,
    Service_Start_Date__c,
    Project_Group__c,
    SOF_MRC__c,
    Address_A__c,
    Address_Z__c
FROM Order
WHERE Status IN ('{status_filter}')
"""

print(f"\n📊 Querying Orders with Status IN {ACTIVE_STATUSES}...")
sf_orders_result = es_sf.query_all(orders_query)
sf_orders_raw = sf_orders_result["records"]
print(f"   Retrieved: {len(sf_orders_raw)} orders")

# Flatten the results
sf_orders = []
for order in sf_orders_raw:
    sf_orders.append(
        {
            "SF_Order_Id": order["Id"],
            "SF_Order_Name": order.get("Name"),
            "Service_ID__c": order.get("Service_ID__c"),
            "SF_Status": order["Status"],
            "AccountId": order.get("AccountId"),
            "Account_Name": order["Account"]["Name"] if order.get("Account") else None,
            "Billing_Invoice__c": order.get("Billing_Invoice__c"),
            "BAN_Name": (
                order["Billing_Invoice__r"]["Name"]
                if order.get("Billing_Invoice__r")
                else None
            ),
            "OSS_Order__c": order.get("OSS_Order__c"),
            "Service_Start_Date__c": order.get("Service_Start_Date__c"),
            "Project_Group__c": order.get("Project_Group__c"),
            "SOF_MRC__c": order.get("SOF_MRC__c"),
            "Address_A__c": order.get("Address_A__c"),
            "Address_Z__c": order.get("Address_Z__c"),
        }
    )

sf_orders_df = pd.DataFrame(sf_orders)
print(f"\n✅ Total SF Orders with Active status: {len(sf_orders_df)}")


STEP 1: QUERYING ES SALESFORCE ORDERS

📊 Querying Orders with Status IN ['Activated', 'Disconnect in Progress', 'Suspended (Late Payment)']...
   Retrieved: 17972 orders

✅ Total SF Orders with Active status: 17972


In [6]:
# === STEP 2: APPLY EXCLUSIONS ===

print("\n" + "=" * 80)
print("STEP 2: APPLYING EXCLUSIONS")
print("=" * 80)

# Count before exclusion
total_before = len(sf_orders_df)

# Identify PA MARKET DECOM orders
pa_decom_mask = (
    sf_orders_df["Project_Group__c"]
    .fillna("")
    .str.contains(EXCLUDE_PROJECT_GROUP, case=False)
)
excluded_pa_decom = sf_orders_df[pa_decom_mask].copy()
print(f"\n❌ Excluded (PA MARKET DECOM): {len(excluded_pa_decom)}")

# Filter to in-scope orders
sf_orders_df = sf_orders_df[~pa_decom_mask].copy()
print(f"✅ In-scope orders: {len(sf_orders_df)}")


STEP 2: APPLYING EXCLUSIONS

❌ Excluded (PA MARKET DECOM): 887
✅ In-scope orders: 17085


In [7]:
# === STEP 3: ANALYZE OSS_Order__c POPULATION ===

print("\n" + "=" * 80)
print("STEP 3: ANALYZING OSS_Order__c POPULATION")
print("=" * 80)

has_oss_id = sf_orders_df["OSS_Order__c"].notna() & (sf_orders_df["OSS_Order__c"] != "")
orders_with_oss = sf_orders_df[has_oss_id].copy()
orders_without_oss = sf_orders_df[~has_oss_id].copy()

print(f"\n📊 OSS_Order__c Population:")
print(
    f"   With OSS_Order__c: {len(orders_with_oss)} ({100*len(orders_with_oss)/len(sf_orders_df):.1f}%)"
)
print(
    f"   Without OSS_Order__c: {len(orders_without_oss)} ({100*len(orders_without_oss)/len(sf_orders_df):.1f}%)"
)

# Get unique OSS order IDs to query
oss_order_ids = orders_with_oss["OSS_Order__c"].dropna().astype(int).unique().tolist()
print(f"\n   Unique OSS Order IDs to query: {len(oss_order_ids)}")


STEP 3: ANALYZING OSS_Order__c POPULATION

📊 OSS_Order__c Population:
   With OSS_Order__c: 11681 (68.4%)
   Without OSS_Order__c: 5404 (31.6%)

   Unique OSS Order IDs to query: 11673


In [8]:
# === STEP 4: QUERY OSS ORDERS ===

print("\n" + "=" * 80)
print("STEP 4: QUERYING OSS ORDERS")
print("=" * 80)

if len(oss_order_ids) > 0:
    # Query in chunks to avoid query size limits
    chunk_size = 5000
    oss_orders = []

    for i in range(0, len(oss_order_ids), chunk_size):
        chunk = oss_order_ids[i : i + chunk_size]
        ids_str = ",".join(str(x) for x in chunk)

        oss_query = f"""
        SELECT 
            order_id,
            order_state_cd,
            order_type_cd,
            bill_start_date,
            bill_end_date,
            circuit_active_date,
            account_id,
            service_id
        FROM om.orders
        WHERE order_id IN ({ids_str})
        """

        with oss_conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(oss_query)
            rows = cur.fetchall()
            oss_orders.extend([dict(row) for row in rows])

        print(f"   Chunk {i//chunk_size + 1}: Retrieved {len(rows)} orders")

    oss_orders_df = pd.DataFrame(oss_orders)
    print(f"\n✅ Total OSS orders retrieved: {len(oss_orders_df)}")

    # Check for OSS IDs not found
    found_ids = set(oss_orders_df["order_id"].tolist())
    not_found_ids = set(oss_order_ids) - found_ids
    print(f"   OSS Order IDs not found in OSS: {len(not_found_ids)}")
else:
    oss_orders_df = pd.DataFrame()
    not_found_ids = set()
    print("⚠️ No OSS Order IDs to query")


STEP 4: QUERYING OSS ORDERS
   Chunk 1: Retrieved 4956 orders
   Chunk 2: Retrieved 4956 orders
   Chunk 3: Retrieved 1673 orders

✅ Total OSS orders retrieved: 11585
   OSS Order IDs not found in OSS: 88


In [9]:
# === STEP 5: JOIN SF AND OSS DATA ===

print("\n" + "=" * 80)
print("STEP 5: JOINING SF AND OSS DATA")
print("=" * 80)

# Prepare for join
orders_with_oss["OSS_Order_ID"] = orders_with_oss["OSS_Order__c"].astype(int)

if len(oss_orders_df) > 0:
    # Merge SF orders with OSS data
    merged_df = orders_with_oss.merge(
        oss_orders_df, left_on="OSS_Order_ID", right_on="order_id", how="left"
    )

    # Add OSS state description
    merged_df["OSS_State_Desc"] = merged_df["order_state_cd"].map(OSS_ORDER_STATES)

    print(f"\n✅ Merged records: {len(merged_df)}")
    print(f"   Records with OSS match: {merged_df['order_id'].notna().sum()}")
    print(f"   Records without OSS match: {merged_df['order_id'].isna().sum()}")
else:
    merged_df = orders_with_oss.copy()
    merged_df["order_state_cd"] = None
    merged_df["OSS_State_Desc"] = None
    merged_df["bill_start_date"] = None
    merged_df["bill_end_date"] = None


STEP 5: JOINING SF AND OSS DATA

✅ Merged records: 11681
   Records with OSS match: 11593
   Records without OSS match: 88


In [10]:
# === STEP 6: DETERMINE ACTIVELY BILLING STATUS ===

print("\n" + "=" * 80)
print("STEP 6: DETERMINING ACTIVELY BILLING STATUS")
print("=" * 80)

today = pd.Timestamp.now().normalize()
print(f"\nToday's date for comparison: {today.date()}")

# Convert date columns
if "bill_start_date" in merged_df.columns:
    merged_df["bill_start_date"] = pd.to_datetime(
        merged_df["bill_start_date"], utc=True
    ).dt.tz_localize(None)
    merged_df["bill_end_date"] = pd.to_datetime(
        merged_df["bill_end_date"], utc=True
    ).dt.tz_localize(None)


# Determine actively billing
# Logic: CL state AND bill_start_date <= today AND (bill_end_date IS NULL OR bill_end_date > today)
def is_actively_billing(row):
    if pd.isna(row.get("order_state_cd")):
        return "No OSS Match"

    state = row["order_state_cd"].strip() if row["order_state_cd"] else None
    bill_start = row.get("bill_start_date")
    bill_end = row.get("bill_end_date")

    if state != "CL":
        return f"Not Closed ({state})"

    if pd.isna(bill_start):
        return "CL - No Bill Start Date"

    if bill_start > today:
        return "CL - Bill Start Future"

    if pd.notna(bill_end) and bill_end <= today:
        return "CL - Bill End Passed"

    return "ACTIVELY BILLING"


merged_df["Billing_Status"] = merged_df.apply(is_actively_billing, axis=1)

# Summary
print("\n📊 Billing Status Distribution:")
billing_status_counts = merged_df["Billing_Status"].value_counts()
for status, count in billing_status_counts.items():
    pct = 100 * count / len(merged_df)
    print(f"   {status}: {count} ({pct:.1f}%)")


STEP 6: DETERMINING ACTIVELY BILLING STATUS

Today's date for comparison: 2026-01-07

📊 Billing Status Distribution:
   ACTIVELY BILLING: 11487 (98.3%)
   No OSS Match: 88 (0.8%)
   CL - Bill End Passed: 78 (0.7%)
   Not Closed (OA): 9 (0.1%)
   Not Closed (CA): 9 (0.1%)
   CL - Bill Start Future: 6 (0.1%)
   CL - No Bill Start Date: 4 (0.0%)


In [11]:
# === STEP 7: BUILD COMPLETE ANALYSIS ===

print("\n" + "=" * 80)
print("STEP 7: BUILDING COMPLETE ANALYSIS")
print("=" * 80)

# Orders without OSS ID need categorization too
orders_without_oss["Billing_Status"] = "No OSS_Order__c in SF"
orders_without_oss["order_state_cd"] = None
orders_without_oss["OSS_State_Desc"] = None
orders_without_oss["bill_start_date"] = None
orders_without_oss["bill_end_date"] = None

# Combine all in-scope orders
# Select common columns
common_cols = [
    "SF_Order_Id",
    "SF_Order_Name",
    "Service_ID__c",
    "SF_Status",
    "AccountId",
    "Account_Name",
    "Billing_Invoice__c",
    "BAN_Name",
    "OSS_Order__c",
    "Service_Start_Date__c",
    "Project_Group__c",
    "SOF_MRC__c",
    "Address_A__c",
    "Address_Z__c",
    "order_state_cd",
    "OSS_State_Desc",
    "bill_start_date",
    "bill_end_date",
    "Billing_Status",
]

# Ensure columns exist in both dataframes
for col in common_cols:
    if col not in merged_df.columns:
        merged_df[col] = None
    if col not in orders_without_oss.columns:
        orders_without_oss[col] = None

all_orders_df = pd.concat(
    [merged_df[common_cols], orders_without_oss[common_cols]], ignore_index=True
)

print(f"\n✅ Total in-scope orders analyzed: {len(all_orders_df)}")


STEP 7: BUILDING COMPLETE ANALYSIS

✅ Total in-scope orders analyzed: 17085


C:\Users\vjero\AppData\Local\Temp\ipykernel_14016\1244295389.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_orders_df = pd.concat(
C:\Users\vjero\AppData\Local\Temp\ipykernel_14016\1244295389.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_orders_df = pd.concat(


In [12]:
# === STEP 8: GENERATE SUMMARY STATISTICS ===

print("\n" + "=" * 80)
print("STEP 8: SUMMARY STATISTICS")
print("=" * 80)

summary_data = []

# Overall counts
summary_data.append(("SCOPE", "Total SF Orders with Active Status", total_before, ""))
summary_data.append(
    ("SCOPE", "Excluded (PA MARKET DECOM)", len(excluded_pa_decom), "Not migrating")
)
summary_data.append(("SCOPE", "In-Scope Orders", len(all_orders_df), ""))
summary_data.append(("", "", "", ""))

# OSS Linkage
summary_data.append(
    (
        "OSS LINKAGE",
        "Orders with OSS_Order__c",
        len(orders_with_oss),
        "Can verify in OSS",
    )
)
summary_data.append(
    (
        "OSS LINKAGE",
        "Orders without OSS_Order__c",
        len(orders_without_oss),
        "Cannot verify - no OSS link",
    )
)
summary_data.append(
    (
        "OSS LINKAGE",
        "OSS IDs not found in OSS",
        len(not_found_ids),
        "SF has ID but OSS has no record",
    )
)
summary_data.append(("", "", "", ""))

# Billing Status breakdown
summary_data.append(("BILLING STATUS", "", "", ""))
for status, count in all_orders_df["Billing_Status"].value_counts().items():
    pct = f"{100*count/len(all_orders_df):.1f}%"
    summary_data.append(("", status, count, pct))

summary_data.append(("", "", "", ""))

# Actively billing summary
actively_billing = all_orders_df[all_orders_df["Billing_Status"] == "ACTIVELY BILLING"]
summary_data.append(
    (
        "MIGRATION SCOPE",
        "Confirmed Actively Billing (migrate)",
        len(actively_billing),
        "OSS confirms active billing",
    )
)
summary_data.append(
    (
        "MIGRATION SCOPE",
        "Not Actively Billing or Unknown",
        len(all_orders_df) - len(actively_billing),
        "Review before migrating",
    )
)

summary_df = pd.DataFrame(
    summary_data, columns=["Category", "Metric", "Count", "Notes"]
)

# Print summary
print("\n" + "=" * 60)
print("MIGRATION SCOPE SUMMARY")
print("=" * 60)
print(f"\nTotal In-Scope SF Orders: {len(all_orders_df):,}")
print(f"Confirmed Actively Billing: {len(actively_billing):,}")
print(f"Not Confirmed / Unknown: {len(all_orders_df) - len(actively_billing):,}")


STEP 8: SUMMARY STATISTICS

MIGRATION SCOPE SUMMARY

Total In-Scope SF Orders: 17,085
Confirmed Actively Billing: 11,487
Not Confirmed / Unknown: 5,598


In [13]:
# === STEP 9: CROSS-TAB ANALYSIS ===

print("\n" + "=" * 80)
print("STEP 9: CROSS-TAB ANALYSIS")
print("=" * 80)

# SF Status vs OSS State
print("\n📊 SF Status vs OSS State:")
crosstab = pd.crosstab(
    all_orders_df["SF_Status"],
    all_orders_df["order_state_cd"].fillna("NO OSS"),
    margins=True,
)
print(crosstab)

# SF Status vs Billing Status
print("\n📊 SF Status vs Billing Status:")
crosstab2 = pd.crosstab(
    all_orders_df["SF_Status"], all_orders_df["Billing_Status"], margins=True
)
print(crosstab2)


STEP 9: CROSS-TAB ANALYSIS

📊 SF Status vs OSS State:
order_state_cd            CA     CL  NO OSS  OA    All
SF_Status                                             
Activated                  9  11377    5488   9  16883
Disconnect in Progress     0    197       4   0    201
Suspended (Late Payment)   0      1       0   0      1
All                        9  11575    5492   9  17085

📊 SF Status vs Billing Status:
Billing_Status            ACTIVELY BILLING  CL - Bill End Passed  \
SF_Status                                                          
Activated                            11289                    78   
Disconnect in Progress                 197                     0   
Suspended (Late Payment)                 1                     0   
All                                  11487                    78   

Billing_Status            CL - Bill Start Future  CL - No Bill Start Date  \
SF_Status                                                                   
Activated           

In [14]:
# === STEP 10: EXPORT TO EXCEL ===

print("\n" + "=" * 80)
print("STEP 10: EXPORTING TO EXCEL")
print("=" * 80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_sf_oss_order_analysis_{timestamp}.xlsx"

# Styles
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="1F4E79")
alt_fill = PatternFill("solid", fgColor="F2F2F2")
green_fill = PatternFill("solid", fgColor="C6EFCE")
red_fill = PatternFill("solid", fgColor="FFC7CE")
yellow_fill = PatternFill("solid", fgColor="FFEB9C")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

wb = Workbook()

# --- Sheet 1: Summary ---
ws_summary = wb.active
ws_summary.title = "Summary"

ws_summary["A1"] = "ES Salesforce vs OSS Order Analysis"
ws_summary["A1"].font = Font(bold=True, size=16)
ws_summary["A2"] = f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

for r_idx, row in enumerate(
    dataframe_to_rows(summary_df, index=False, header=True), start=4
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_summary.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 4:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

ws_summary.column_dimensions["A"].width = 18
ws_summary.column_dimensions["B"].width = 45
ws_summary.column_dimensions["C"].width = 12
ws_summary.column_dimensions["D"].width = 35

print(f"   ✅ Summary sheet created")

# --- Sheet 2: Actively Billing ---
ws_active = wb.create_sheet("Actively_Billing")
actively_billing_export = actively_billing[
    [
        "SF_Order_Id",
        "Service_ID__c",
        "SF_Status",
        "Account_Name",
        "BAN_Name",
        "OSS_Order__c",
        "order_state_cd",
        "bill_start_date",
        "bill_end_date",
        "Service_Start_Date__c",
        "SOF_MRC__c",
    ]
].copy()

for r_idx, row in enumerate(
    dataframe_to_rows(actively_billing_export, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_active.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ Actively_Billing sheet created ({len(actively_billing_export)} records)")

# --- Sheet 3: Not Actively Billing ---
ws_not_active = wb.create_sheet("Not_Actively_Billing")
not_actively_billing = all_orders_df[
    all_orders_df["Billing_Status"] != "ACTIVELY BILLING"
][
    [
        "SF_Order_Id",
        "Service_ID__c",
        "SF_Status",
        "Account_Name",
        "BAN_Name",
        "OSS_Order__c",
        "order_state_cd",
        "OSS_State_Desc",
        "bill_start_date",
        "bill_end_date",
        "Billing_Status",
        "SOF_MRC__c",
    ]
].copy()

for r_idx, row in enumerate(
    dataframe_to_rows(not_actively_billing, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_not_active.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ Not_Actively_Billing sheet created ({len(not_actively_billing)} records)")

# --- Sheet 4: All Orders Detail ---
ws_all = wb.create_sheet("All_Orders")

for r_idx, row in enumerate(
    dataframe_to_rows(all_orders_df, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_all.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ All_Orders sheet created ({len(all_orders_df)} records)")

# --- Sheet 5: No OSS Link ---
ws_no_oss = wb.create_sheet("No_OSS_Link")
no_oss_export = orders_without_oss[
    [
        "SF_Order_Id",
        "Service_ID__c",
        "SF_Status",
        "Account_Name",
        "BAN_Name",
        "Service_Start_Date__c",
        "SOF_MRC__c",
    ]
].copy()

for r_idx, row in enumerate(
    dataframe_to_rows(no_oss_export, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_no_oss.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ No_OSS_Link sheet created ({len(no_oss_export)} records)")

# --- Sheet 6: Excluded PA DECOM ---
ws_excluded = wb.create_sheet("Excluded_PA_DECOM")
excluded_export = excluded_pa_decom[
    [
        "SF_Order_Id",
        "Service_ID__c",
        "SF_Status",
        "Account_Name",
        "Project_Group__c",
        "SOF_MRC__c",
    ]
].copy()

for r_idx, row in enumerate(
    dataframe_to_rows(excluded_export, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_excluded.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ Excluded_PA_DECOM sheet created ({len(excluded_export)} records)")

# --- Sheet 7: OSS State Crosstab ---
ws_crosstab = wb.create_sheet("SF_vs_OSS_Crosstab")
crosstab_reset = crosstab.reset_index()

for r_idx, row in enumerate(
    dataframe_to_rows(crosstab_reset, index=False, header=True), start=1
):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_crosstab.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
        cell.border = thin_border

print(f"   ✅ SF_vs_OSS_Crosstab sheet created")

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel file saved: {output_file}")


STEP 10: EXPORTING TO EXCEL
   ✅ Summary sheet created
   ✅ Actively_Billing sheet created (11487 records)
   ✅ Not_Actively_Billing sheet created (5598 records)
   ✅ All_Orders sheet created (17085 records)
   ✅ No_OSS_Link sheet created (5404 records)
   ✅ Excluded_PA_DECOM sheet created (887 records)
   ✅ SF_vs_OSS_Crosstab sheet created

✅ Excel file saved: es_sf_oss_order_analysis_20260107_174442.xlsx


In [15]:
# === CLEANUP ===

print("\n" + "=" * 80)
print("CLEANUP")
print("=" * 80)

oss_conn.close()
print("✅ OSS connection closed")

print(f"\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print(f"\nOutput file: {output_file}")
print(f"\nKey findings:")
print(f"  - Total in-scope orders: {len(all_orders_df):,}")
print(f"  - Confirmed actively billing: {len(actively_billing):,}")
print(
    f"  - Not confirmed / review needed: {len(all_orders_df) - len(actively_billing):,}"
)


CLEANUP
✅ OSS connection closed

ANALYSIS COMPLETE

Output file: es_sf_oss_order_analysis_20260107_174442.xlsx

Key findings:
  - Total in-scope orders: 17,085
  - Confirmed actively billing: 11,487
  - Not confirmed / review needed: 5,598
